**MainFlow**

entry

In [ ]:
#set($callId = $params.callId)
#if(!$phone)
 #set($phone = $params.get("session.callerid"))
#end

#set($voiceFileName = "hello_welcome_ai")
`$voiceFileName`

entry-handler

In [ ]:
#set($params.playedMessages = 'Entry-handler')
::::::::::::::::::::::::::::::::
::::: MainFlow params logger::::
phone:::::::::::: $phone
session.callid::: $params.get("session.callerid")
::::::::::::::::::::::::::::::::
params::::::::::: $params
messages::::::::: $messages
::::::::::::::::::::::::::::::::
::::personId::::::::::::::: $personId
::::actorType:::::::::::::: $actorType
::::actorId:::::::::::::::: $actorId
::::refKey::::::::::::::::: $refKey::::messagesscheduledServicegroupId::::phone::::::::::::::::::$phone
::::internal:::::::::::::::internal:::causes exception that hides other watch logs
::::callId = $internal.callId
::::::::::::::::::::::::::::::::

#set($callId = $internal.callId)
#set($pinAttempts = 0)
#set($maxPinAttempt = 3)
#set($isLocaleSet = "NO")
  
#if($refKey)
	#set($sourceId = $refKey)
  #set($sourceType = "SCHEDULED_SERVICE_GROUP")
  #set($serviceGroup = $messagingGroupService.getById($Integer.parseInt($refKey)))
  
  ::::personId::::::::::::::: $personId
::::actorType:::::::::::::: $actorType
::::actorId:::::::::::::::: $actorId
  

  ##set($patientId = $serviceGroup.getPatient().personId)
patientId:::::::: $patientId
service group:::: $sourceId
  ## testing|main
#else
	#set($sourceId = $callId)
  #set($sourceType = "OTHER")
  #set($patientId = $actorId)
#end

#if(!$actorId && $personId)
	#set ($actorId = $personId)
#elseif($actorId)	
	#set ($actorId = $Integer.parseInt($actorId)) 
#end

Testing actorId:::::::::: $actorId
#if($actorId)
	#set($person = $personDAO.getPerson($actorId))
#else
 |main|
#end 


#if($person)

	  #set($personStatus = "DEACTIVATED")
    #set($maxPinAttempt = 3)
    #set($pinAttempts = 0)
  	#foreach($attribute in $person.getActiveAttributes())

      #if($attribute.attributeType.name == 'Person status' && !$attribute.voided)
      	#set($personStatus = $attribute.value)
      #end
      #if($attribute.attributeType.name == 'personLanguage' && !$attribute.voided)
        #set($locale = $attribute.value)
      	#set($isLocaleSet = "YES")
        #set($isvoided = $attribute.voided)
      #end
      ##if($attribute.attributeType.name == 'personLanguage' && !$attribute.voided)
      	##set($locale = $attribute.value)
      ##end
    #end
:::::::::Locale:::::::::::::: $locale
:::::::::isLocaleSet::::::::: $isLocaleSet
:::::::::PersonStatus:::::::: $personStatus
:::::::::isvoided?::::::::::: $isvoided

phone:::::::::::: $phone
##session.callid::: $params.get("session.callerid")
::::::::::::::::::::::::::::::::
##params::::::::::: $params
##messages::::::::: $messages
::::::::::::::::::::::::::::::::
personId::::::::: $personId
actorType:::::::: $actorType
actorId:::::::::: $actorId
refKey::::::::::: $refKey
internal::::::::: $internal
::::::::::::::::::::::::::::::::
		|MainFlow.main|
#else
		|exit|
#end

main

In [ ]:
#set($voiceFileName = "visit_reminder_intro")
`$voiceFileName`

main-handler

In [ ]:
#set($params.playedMessages = 'MainFlow_main')
#set($noInputCount = 0)
#set($invalidCount = 0)

::::check local here:::: If correct then continue to outgoing flow
#if($isLocaleSet == "YES") 
:::: locale set:::: 


  #if($internal.callDirection == "OUTGOING")
    #if($messages.size() == 0)
      ::::Go to outgoing flow whether message exists/not 
      ::::This allows "initiate call" button to move call on to outgoingflow
      :::: outging message size is zero!:::: 
      |OutgoingFlow.entry|
      :::: may fail because outging message size is zero!:::: 
    #else 
      ::::message size is greater than zero:::::::::::::::::::::::::
      :::: outoging message size is greater than 0!:::: 
      |OutgoingFlow.entry|
    #end
  #else
    ::::TODO:::::incoming call:::
  #end
#else
		|goodbye|
#end 

exit

#set($params.playedMessages = 'MainFlow_exit')

no-input

no-input-handler

In [ ]:
#set($params.playedMessages = 'MainFlow_no-input')
#if ($noInputCount < 2)
	#set($noInputCount = $noInputCount + 1)
	|$returnNode|
#else
	|OutgoingFlow.remove-first-message|
#end

goodbye

In [ ]:
#set($voiceFileName = "goodbye")
`$voiceFileName`

goodbye-handler

In [ ]:
#set($params.playedMessages = 'Mainflow_goodbye')
|exit|

**OutgoingFlow**

OutgoingFlow entry

In [ ]:
#set($voiceFileName = "remind_you_about")
`$voiceFileName`

OutgoingFlow entry-handler

In [ ]:
##-- Shows where we are
#set($params.playedMessages = 'OutgoingFlow_entry-handler')

:::Outgoingflow::::Entry
:::MessagesSize::::::$messages.size()
:::playedMessages::::::::::::$params.playedMessages

#if($messages.size() > 0)
 #set($params.playedMessages = $params.playedMessages + '|OutgoingFlow_messages_greater_than_0')
	## Get variables out of message
 #set($message = $messages.get(0))
 #set($messageId = $messages.get(0).messageId)
 ::::messageId :::::::::::::::$messageId
 ::::date ::::::::::::::::::$DateUtil.now()
 ::::call ::::::::::::::::::$callId
 ::::phone::::::::::::::::::$phone
 ::::personId::::::::::::::: $personId
 ::::actorType:::::::::::::: $actorType
 ::::actorId:::::::::::::::: $actorId
 ::::refKey::::::::::::::::: $refKey::::messagesscheduledServicegroupId
 ##::::internal:::::::::::::::internal:::causes exception that hides other watch logs
 ##::::callId = $internal.callId

 	## Set message variables to callflow variables
 	#if($refKey)
 
    #set($sourceId = $refKey)
    #set($sourceType = "SCHEDULED_SERVICE_GROUP")
    #set($serviceGroup = $messagingGroupService.getById($Integer.parseInt($refKey)))
    #set($patientId = $serviceGroup.getPatient().personId)
		::::sourceType:::::::::::::: $sourceType
    #set($params.playedMessages = $params.playedMessages + '|refkey')
  
	#else
		#set($sourceId = $callId)
  	#set($sourceType = "OTHER")
  	#set($patientId = $actorId)
		::::sourceType:::::::::::::: $sourceType
    #set($params.playedMessages = $params.playedMessages + '|no_refkey')
	#end
   
	#if($message.name == 'Visit reminder' || $message.name == 'Welcome Message')
  :::Visit Reminder flow is here????!!!:::::::::::
    #set($params.playedMessages = $params.playedMessages + '|message_name_visit_reminder')
    |VisitReminderFlow.|
 	#end 
  :::ARE WE here????!!!:::::::::::
  |MainFlow.goodbye|
  
  :::::after  removal messages::::::::: $messages.size()
  :::::after removal messages:::::::::$message.name
#else 
:::Messages less than Zero:::::::::::
  #set($params.playedMessages = $params.playedMessages + '|OutgoingFlow_messages_less_than_0')
  |MainFlow.goodbye|
#end

**VisitReminderFlow**

VisitReminderFlow entry

In [ ]:
#set($voiceFileName = "listen_carefully_to_hear")
`$voiceFileName`

VisitReminderFlow entry-handler

#set($params.playedMessages = $voiceFileName + '|' + $params.userInput)
:::VisistReminderFlow::::Entry
::::::::::::::TESTING :::::::::::
#set($textToRead = "You have scheduled visit")
#set($locationId = $messages.get(0).params.locationId)
#set($visitTime = $messages.get(0).params.timeStarted)
#set($visitDate = $messages.get(0).params.dateStarted) 
#set($visitDateWithoutTime = $visitDate.substring(0, 10)) 
#set($simpleDateFormat = $dateTool.getSimpleDateFormat("yyyy-MM-dd")) 
#set($date = $simpleDateFormat.parse($visitDateWithoutTime))  
#set($dayOfWeekFormat = $dateTool.getSimpleDateFormat("EEEE")) 
#set($dayOfWeek = $dayOfWeekFormat.format($date))
 
#set($splitByDash = $visitDateWithoutTime.split("-"))
#set($visitYear = $splitByDash.get(0))
#set($visitMonth = $splitByDash.get(1)) 
#set($visitDay = $splitByDash.get(2)) 

#set($locationToRead = "")   
#set($timeToRead = "")
#if($locationId)
::::::::::::::TESTING BUG HERE??
	#set($locationId = $Integer.parseInt($locationId))
  #set($location = $locationService.getLocation($locationId).getDisplayString())
	#set($locationToRead = "Please go to the clinic $location")
#end
#if($visitTime)
  #set($timeToRead = "at $visitTime")
#end

 	:::::::::: VISIT DATE TESTING ::::::::::
	:::::::::: Visit Time ::::::::::$visitTime
  :::::::::: Visit Date ::::::::::$visitDate
  :::::::::: Visit Date ::::::::::$visitDateWithoutTime
  :::::::::: Visit Year ::::::::::$visitYear
  :::::::::: Visit Month :::::::::$visitMonth
  :::::::::: Visit Day :::::::::::$visitDay
  :::::::::: Visit DayOfWeek:::::::::::$dayOfWeek

|play-reminder|

play-reminder

In [ ]:
#set($voiceFileName = "next_visit_date_intro")
`$voiceFileName`

#set($voiceFileName = "visit_day_$visitDay")
`$voiceFileName`

#set($voiceFileName = "visit_month_$visitMonth")
`$voiceFileName` 

#set($voiceFileName = "visit_year_$visitYear")
`$voiceFileName`

play-reminder-handler

In [ ]:
##set($params.playedMessages = 'play-reminder-handler')

:::::::::: Begin REMINDER check::::::::::
#if($reminderPlayed)
	:::::::::: REPEATED REMINDER ::::::::::
	:::::::::: PARAMS ::::::::::$params
  #set($params.playedMessages = 'play-reminder-repeat')
	|repeat-message|
#else
  #set($params.playedMessages = 'play-reminder-no-repeat')
	:::::::Here WE ARE in play!!!:::::::::::$message.name
 ## |HealthTipFlow.|
  :::Here WE ARE in play!!!:::::::::::$message.name:::::: NOT REPEATED REMINDER ::::::::::
  ## Working version 
  ## $messagesService.registerAttempt($visitReminderMessageId, 'DELIVERED', $DateUtil.now(), $callId)
   $messagesService.registerAttempt($messageId, 'DELIVERED', $DateUtil.now(), $callId)

  #set($reminderPlayed = true)
    
  :::::::::: Visit Time ::::::::::$visitTime
  :::::::::: Visit Date ::::::::::$visitDate
 	:::::::::: Visit Date without time ::::::::::visitDateWithoutTime
  
#end
|end|

end

end-handler

In [ ]:
#set($params.playedMessages = 'VisitReminderFlow_end')
##$messages.remove(0)
##|OutgoingFlow.entry-handler|
|MainFlow.goodbye|